# Bayesian Statistics

## Indicator Functions
An indicator function, sometimes known as a Heaviside or unit-step function, takes a value evaluates it against some condition, returning either 1 (the condition is true), or 0 (the condition if false):

$$
I_{\{x>3\}}(x) =
    \begin{cases}
        0 & \text{if } x \leq 3 \\
        1 & \text{if } x > 3
    \end{cases}
$$

In [19]:
# an example indicator function
Ia <- function(x) {
    if (x <= 3) 0
    else 1
}

# here we create a randomised vector of 12 dice rolls:
xs <- sample(6, 12, replace=TRUE)
xs

# now we apply the indicator function to see which ones 
# are greater than 3:
gt3 <- sapply(xs, Ia)
gt3

# summary info
mean(xs)
median(xs)
var(xs)
sd(xs)

[1] 3 3 6 6 3 3 4 4 3 1 5 3

[1] 0 0 1 1 0 0 1 1 0 0 1 0

[1] 3.666667

[1] 3

[1] 2.060606

[1] 1.435481

Indicator functions can be strung together as part of a larger function. In this example, what would be the result of $g(x)$ where $x=18$?

$$g(x) = ( 2000 \times I_{\{x<2\}}(x) ) + ( 200x \times I_{\{x=5\}}(x) ) + ( 2x \times I_{\{x>10\}}(x) )$$

Here, the first indicator function evaluates to 0, since 16 is greater than 2.
The second indicator function evaluates to 0 also.
The third indicator function evaluates to 2x16 (since 16 is greater than 10), so the final result is 32.

## Bernoulli Distributions
The simplest way to think about a Bernoulli distribuion is as a function that simulates a coin toss. By default the coin is a fair one (the probability of heads is 0.5). I you want to make it an unfair coin you can vary the probability.
We say that a random variable X follows a Bernoulli distribution with probability p:

$$X \sim B(p)$$

In [20]:
# Here's two functions, one that shows a simple Bernoulli function 
# that returns '1' (success) or '0' failure:
B <- function(p=0.5) {
    probs <- c(p, 1-p)
    sample(c(1, 0), 1, replace = TRUE, probs) 
}

# And a similar one that simulates the toss of a coin:
coinFlip <- function(p=0.5) {
    probs <- c(p, 1-p)
    sample(c('H', 'T'), 1, replace = TRUE, probs)
}

In fact a Bernoulli function is written like this:

$$f(x|p) = p^{x}(1-p)^{1-x}$$

which is equivalent to the indicator function:

$$I_{\{x \in \{0,1\}\}}(x)$$

In [21]:
B()

# simulate a biased coin
# should return 'H' in 55% of the time
coinFlip(0.55)

[1] 0

[1] "H"

## Binomial Distributions
The generalisation of the Bernoulli distribution, when we have $n$ repeated Bernoullis, is the binomial distribution.
For a distribution to be binomial, it must:

* Have two potential outcomes per trial (success or failure)
* The probability of success (p) must remain the same for each trial
* The number of trials (n) is fixed
* Each trial is independent

In [22]:
seriesOfFlips <- function(n) {
    flips <- character(n)
    for (c in seq_along(flips)){
        flips[c] <- coinFlip()
    }
    return(flips)
}

# Run a series of 5 flips, and repeat a few times...
seriesOfFlips(5)

[1] "T" "T" "T" "H" "T"

With the binomial distribution, we get the sum of n Bernoullis:

$$X \sim Bin(n, p)$$

Which is written with a probability function of:

$$f(x|p) = \binom{n}{x}p^{x}(1-p)^{n-x}$$

Alternatively it can be expressed in terms of probabilities, like this:

$$P(X = x) = {^nC_x}p^x(1-p)^{n-x}$$

The important thing to know here is that $\binom{n}{x}$ and ${^nC_x}$ are equivalent, both mean '$n$ choose $x$'.

## Binomial Coefficients

R has a function `choose` that allows you to find `k` from `n`. 

$$\binom{n}{k} = \frac{n!}{k!(n-k)!}$$

*****

For example to find the coeffient of choose 3 from 4:

$$\binom{4}{3}$$

call the function like this:

```r
choose(4, 3)
```

In [23]:
choose(4, 3)   # How many ways can we select 3 items from 4 items?

[1] 4

In [24]:
choose(5, 2)   # How many ways can we select 2 items from 5 items?

[1] 10

In [25]:
# Probability Mass Function (PMF) for the Binomial distribition
binomialPMF <- function(k, n, p=0.5) {
    choose(n,k) * (p^k) * ((1-p)^(n-k))
}

In [26]:
# the probability of flipping exactly 5 heads in 10 coin tosses is...
binomialPMF(5, 10)

[1] 0.2460938

In [27]:
# we know that 8% of men are colour blind
# what is the probability that, of a sample of 10 men, exactly two are colour blind?

p = 0.08
n = 10
k = 2

binomialPMF(k, n, p)

[1] 0.147807

In [28]:
# and what is the probability that at least 2 men are colour blind?
# in other words P(2 >= X) for X Binomial(10, 0.08)
sum(dbinom(2:11, 10, 0.08))

[1] 0.1878825

In [29]:
# Compute P(45 < X < 55) for X Binomial(100,0.5)
sum(dbinom(46:54, 100, 0.5))

[1] 0.6317984

In [30]:
# probability of pulling exactly one 'E. T. Jaynes' card from 100 cards
binomialPMF(1, 100, 0.0072)

# however, what we really want to know is the probability of getting one or more
# we could sum up the PMFs for all values of k 1-100; an easier way in R is to use
# the `pbinom` function
pbinom(0, 100, 0.0072, lower.tail=FALSE)

# setting the first argument to 0, we are looking at the probability of getting 
# one or more E. T. Jaynes cards. You need to set `lower.tail=FALSE` for this to
# work out, as this  means we want values greater than the first argument.

[1] 0.352085

[1] 0.5145138

## Mean, Variance and Standard Deviation of Binomials

### Expected Value
The Expected Value $E(X)$ of the binomial is the mean of its distribution, which is $n × p$:

$$E(X) = np$$

In our colour blind men example, the exepted number of colour blind men in a sample of 100 men would be:

$$E(X) = 100 \times 0.08 = 8$$

### Variance
The variance is:

$$V(X) = np(1-p)$$

Or, in our example:

$$V(X) = 100 \times 0.08 \times 0.92 = 7.36$$

### Standard Deviation
From the variance, we can calculate the standard deviation as:

$$SD(X) = \sqrt {V(X)}$$

Which, for our example is $\sqrt {7.36}$, or 2.71.

## Exercises

1. What are the parameters of the binomial distribution for the probability of rolling either a 1 or a 20 on a 20-sided die, if we roll the die 12 times?


In [36]:
p <- 0.05 + 0.05      # probability of rolling a 1 or 20 is 0.1
binomialPMF(1, 12, p)

[1] 0.3765727

2. There are four aces in a deck of 52 cards. If you pull a card, return the card, then reshuffle and pull a card again, how many ways can you pull just one ace in five pulls?

In [32]:
choose(5, 1)

# Axxxx
# xAxxx
# xxAxx
# xxxAx
# xxxxA

# probability of pulling 5 aces in 10 pulls is:
p <- 4/52
k <- 5
n <- 10

p5Aces <- binomialPMF(k, n, p)

print(p5Aces)

[1] 5

[1] 0.0004548553


3. When you’re searching for a new job, it’s always helpful to have more than one offer on the table so you can use it in negotiations. If you have a 1/5 probability of receiving a job offer when you interview, and you interview with seven companies in a month, what is the probability you’ll have at least two competing offers by the end of that month?

In [33]:
p <- 0.20
n <- 7  # number of interviews
k <- 2  # number of offers

# this is the probability of receiving just one offer:
binomialPMF(k, n, p)

[1] 0.2752512

In [34]:
# however, we want to find the probabilty of receiving at least one offer:
pbinom(1,n,p,lower.tail = FALSE)

[1] 0.4232832

4. You get a bunch of recruiter emails and find out you have 25 interviews lined up in the next month. Unfortunately, you know this will leave you exhausted, and the probability of getting an offer will drop to 1/10 if you’re tired. You really don’t want to go on this many interviews unless you are at least twice as likely to get at least two competing offers. Are you more likely to get at least two offers if you go for 25 interviews, or stick to just 7?

In [35]:
n <- 25  # number of interviews
p <- 0.10
k <- 2
p.two.or.more.25 <- pbinom(1, n, 1/10, lower.tail = FALSE)

n <- 7   # now just decide on 7 interviews
p.two.or.more.7 <- pbinom(1, n, 1/5, lower.tail = FALSE)

print(p.two.or.more.25)
print(p.two.or.more.7)
print("Go for 25 interviews?")
print(p.two.or.more.25 > p.two.or.more.7)

p.two.or.more.25/p.two.or.more.7

[1] 0.7287941
[1] 0.4232832
[1] "Go for 25 interviews?"
[1] TRUE


[1] 1.721765